# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import datetime as dt
from sql_queries import *

In [2]:
#Connect to the database and get a cursor
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()


In [3]:
#returns list of files in the filepath
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
#get the list of all song data files
song_files = get_files("data/song_data")


In [5]:
#take a random single file
filepath = song_files[0]

In [6]:
df = pd.read_json(filepath, lines=True)
df.head()

,num_songs,artist_id,artist_latitude,artist_longitude,artist_location,artist_name,song_id,title,duration,year
0,1,ARGSJW91187B9B1D6B,35.21962,-80.01955,North Carolina,JennyAnyKind,SOQHXMF12AB0182363,Young Boy Blues,218.77506,0


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [7]:
#get the required columns for the song data
song_data = df[["song_id","title", "artist_id", "year", "duration"]]

#get the first row values in the song data
song_data = song_data.values[0]


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [8]:
cur.execute(song_table_insert, song_data)
conn.commit()


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [9]:
#get the required columns for the artist data
artist_data = df[["artist_id","artist_name", "artist_location", "artist_latitude", "artist_longitude"]]

#get the first row values in the artist data
artist_data = artist_data.values[0]


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [10]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [11]:
#get the list of all log data files
log_files = get_files("data/log_data")
print(log_files)

['/home/pranav/Desktop/data/data-modeling-with-postgres/data/log_data/2018/11/2018-11-21-events.json', '/home/pranav/Desktop/data/data-modeling-with-postgres/data/log_data/2018/11/2018-11-17-events.json', '/home/pranav/Desktop/data/data-modeling-with-postgres/data/log_data/2018/11/2018-11-28-events.json', '/home/pranav/Desktop/data/data-modeling-with-postgres/data/log_data/2018/11/2018-11-15-events.json', '/home/pranav/Desktop/data/data-modeling-with-postgres/data/log_data/2018/11/2018-11-10-events.json', '/home/pranav/Desktop/data/data-modeling-with-postgres/data/log_data/2018/11/2018-11-19-events.json', '/home/pranav/Desktop/data/data-modeling-with-postgres/data/log_data/2018/11/2018-11-24-events.json', '/home/pranav/Desktop/data/data-modeling-with-postgres/data/log_data/2018/11/2018-11-23-events.json', '/home/pranav/Desktop/data/data-modeling-with-postgres/data/log_data/2018/11/2018-11-09-events.json', '/home/pranav/Desktop/data/data-modeling-with-postgres/data/log_data/2018/11/2018

In [12]:
#take a single file
filepath = log_files[15]
print(filepath)

/home/pranav/Desktop/data/data-modeling-with-postgres/data/log_data/2018/11/2018-11-07-events.json


In [13]:
df = pd.read_json(filepath, lines=True)
df[50:100]

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
50,None,Logged In,Ava,F,0,Robinson,NaN,free,"New Haven-Milford, CT",GET,Home,1.540932e+12,338,None,200,1541583646796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",50
51,Pixies,Logged In,Ava,F,1,Robinson,89.36444,free,"New Haven-Milford, CT",PUT,NextSong,1.540932e+12,338,Build High,200,1541583683796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",50
52,The Roots / Jack Davey,Logged In,Ava,F,2,Robinson,155.95057,free,"New Haven-Milford, CT",PUT,NextSong,1.540932e+12,338,Atonement,200,1541583772796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",50
53,Mike And The Mechanics,Logged In,Ava,F,3,Robinson,275.12118,free,"New Haven-Milford, CT",PUT,NextSong,1.540932e+12,338,A Beggar On A Beach Of Gold,200,1541583927796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",50
54,Faithless,Logged In,Ava,F,4,Robinson,495.30730,free,"New Haven-Milford, CT",PUT,NextSong,1.540932e+12,338,Music Matters (Mark Knight Dub),200,1541584202796,"""Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebK...",50
55,Kings Of Leon,Logged In,Aiden,M,0,Hess,227.13424,free,"La Crosse-Onalaska, WI-MN",PUT,NextSong,1.540829e+12,204,Be Somebody,200,1541589678796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",86
56,None,Logged In,Sylvie,F,0,Cruz,NaN,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",GET,Home,1.540266e+12,339,None,200,1541590304796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
57,Vader,Logged In,Sylvie,F,1,Cruz,128.91383,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1.540266e+12,339,Carnal,200,1541590661796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
58,Wu-Tang Clan,Logged In,Sylvie,F,2,Cruz,122.46159,free,"Washington-Arlington-Alexandria, DC-VA-MD-WV",PUT,NextSong,1.540266e+12,339,Impossible,200,1541590789796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",10
59,None,Logged In,Ryan,M,0,Smith,NaN,free,"San Jose-Sunnyvale-Santa Clara, CA",GET,Home,1.541017e+12,326,None,200,1541591952796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",26


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [14]:
#filter records where page = 'NextSong'
df = df[(df.page == "NextSong")]

#add a new column "date" coverting timestamp in milliseconds into datetime format
df["date"] = pd.to_datetime(df['ts'])

df


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,date
0,Miami Horror,Logged In,Kate,F,88,Harrell,250.82730,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,Sometimes,200,1541548876796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97,1970-01-01 00:25:41.548876796
1,The White Stripes,Logged In,Kate,F,89,Harrell,241.86730,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,My Doorbell (Album Version),200,1541549126796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97,1970-01-01 00:25:41.549126796
2,Juan Carmona,Logged In,Kate,F,90,Harrell,331.44118,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,Panales de Algodon,200,1541549367796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97,1970-01-01 00:25:41.549367796
3,Alison Krauss / Union Station,Logged In,Kate,F,91,Harrell,171.04934,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,Restless,200,1541549698796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97,1970-01-01 00:25:41.549698796
4,Bullet For My Valentine,Logged In,Kate,F,92,Harrell,235.65016,paid,"Lansing-East Lansing, MI",PUT,NextSong,1.540473e+12,293,Begging For Mercy,200,1541549869796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",97,1970-01-01 00:25:41.549869796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196,The Cat Empire,Logged In,Lily,F,104,Koch,307.46077,paid,"Chicago-Naperville-Elgin, IL-IN-WI",PUT,NextSong,1.541048e+12,221,The Chariot,200,1541624754796,"""Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/5...",15,1970-01-01 00:25:41.624754796
198,STRATOVARIUS,Logged In,Lily,F,1,Burns,350.74567,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,349,Twilight Time,200,1541629614796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32,1970-01-01 00:25:41.629614796
199,The Mantles,Logged In,Lily,F,2,Burns,226.53342,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540621e+12,349,Don't Lie,200,1541629964796,"""Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebK...",32,1970-01-01 00:25:41.629964796
200,Tub Ring,Logged In,Kaleb,M,0,Cook,233.69098,free,"Yuba City, CA",PUT,NextSong,1.540680e+12,213,Invalid,200,1541632356796,Mozilla/5.0 (Windows NT 6.1; WOW64; rv:31.0) G...,54,1970-01-01 00:25:41.632356796


In [15]:
#create a dataframe for time data 

t =pd.DataFrame()

t["ts"] = df["ts"]
t["day"] = df["date"].dt.day
t["hour"] = df["date"].dt.hour
t["week"] = df["date"].dt.week
t["month"] = df["date"].dt.month
t["year"] = df["date"].dt.year
t["weekday"] = df["date"].dt.dayofweek


t.head()


,ts,day,hour,week,month,year,weekday
0,1541548876796,1,0,1,1,1970,3
1,1541549126796,1,0,1,1,1970,3
2,1541549367796,1,0,1,1,1970,3
3,1541549698796,1,0,1,1,1970,3
4,1541549869796,1,0,1,1,1970,3


In [16]:

time_data = t
column_labels = ("ts" , "hour", "day", "week", "month","year", "weekday")


In [17]:
time_df = pd.DataFrame(t,columns =column_labels)
time_df.head()

,ts,hour,day,week,month,year,weekday
0,1541548876796,0,1,1,1,1970,3
1,1541549126796,0,1,1,1,1970,3
2,1541549367796,0,1,1,1,1970,3
3,1541549698796,0,1,1,1,1970,3
4,1541549869796,0,1,1,1,1970,3


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [18]:
#insert time data
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [19]:
#get the required columns for the user data
user_df = df[["userId","firstName", "lastName", "gender", "level"]]
user_df

,userId,firstName,lastName,gender,level
0,97,Kate,Harrell,F,paid
1,97,Kate,Harrell,F,paid
2,97,Kate,Harrell,F,paid
3,97,Kate,Harrell,F,paid
4,97,Kate,Harrell,F,paid
...,...,...,...,...,...
196,15,Lily,Koch,F,paid
198,32,Lily,Burns,F,free
199,32,Lily,Burns,F,free
200,54,Kaleb,Cook,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [20]:
#insert user data
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [21]:
#insert songplays data
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None
        

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [24]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.